In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!unzip -q "/content/drive/Shareddrives/SEAI Project/Dataset/trainingSet.zip" 

In [ ]:
BATCH_SIZE = 64
IMAGE_HEIGHT = 254
IMAGE_WIDTH = 254
SEED = 42
NUM_CLASSES = 2

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input, Dropout

In [ ]:
fpath = "/content/content/training"

In [ ]:
training_set = image_dataset_from_directory(fpath,
                                       labels='inferred',
                                       label_mode='categorical',
                                       color_mode="rgb",
                                       batch_size=BATCH_SIZE,
                                       image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
                                       shuffle=True,
                                       seed=42,
                                       validation_split=0.2,
                                       subset='training',
                                       )

validation_set = image_dataset_from_directory(fpath,
                                       labels='inferred',
                                       label_mode='categorical',
                                       color_mode="rgb",
                                       batch_size=BATCH_SIZE,
                                       image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
                                       shuffle=True,
                                       seed=42,
                                       validation_split=0.2,
                                       subset='validation',
                                       )

Found 2968 files belonging to 2 classes.
Using 2375 files for training.
Found 2968 files belonging to 2 classes.
Using 593 files for validation.


In [ ]:
!unzip -q "/content/drive/Shareddrives/SEAI Project/Dataset/Test.zip" 

In [ ]:
test_set = image_dataset_from_directory("/content/Test",
                                       labels='inferred',
                                       label_mode='categorical',
                                       color_mode="rgb",
                                       batch_size=BATCH_SIZE,
                                       image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
                                       shuffle=False,
                                       )

Found 8617 files belonging to 2 classes.


## Training Inception-v3 frozen + MLP

In [ ]:
input_tensor = Input(shape=(254, 254, 3))

base_model = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(2048, activation='relu')(x)
x = Dropout(0.25)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x)

predictions = Dense(2, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
  layer.trainable = False

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

history = model.fit(
                   training_set,
                    epochs = 20,
                    validation_data = validation_set
                    )

Epoch 1/20
38/38 [==============================] - 42s 621ms/step - loss: 7.6272 - acc: 0.6467 - val_loss: 0.4987 - val_acc: 0.7420
Epoch 2/20
38/38 [==============================] - 20s 510ms/step - loss: 0.5625 - acc: 0.7208 - val_loss: 0.4768 - val_acc: 0.7116
Epoch 3/20
38/38 [==============================] - 20s 508ms/step - loss: 0.4486 - acc: 0.7651 - val_loss: 0.4462 - val_acc: 0.7673
Epoch 4/20
38/38 [==============================] - 20s 508ms/step - loss: 0.4247 - acc: 0.7819 - val_loss: 0.4308 - val_acc: 0.7673
Epoch 5/20
38/38 [==============================] - 20s 507ms/step - loss: 0.3976 - acc: 0.7966 - val_loss: 0.4224 - val_acc: 0.7993
Epoch 6/20
38/38 [==============================] - 20s 506ms/step - loss: 0.4153 - acc: 0.7869 - val_loss: 0.4565 - val_acc: 0.7454
Epoch 7/20
38/38 [==============================] - 20s 506ms/step - loss: 0.4168 - acc: 0.7949 - val_loss: 0.4710 - val_acc: 0.7723
Epoch 8/20
38/38 [==============================] - 20s 505ms/step - 

## Finetuning Inception-v3

In [ ]:
for layer in model.layers[:249]:
  layer.trainable = False

for layer in model.layers[249:]:
  layer.trainable = True

#Recompile the model for these modifications to take effect

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(
                    training_set,
                    epochs = 10,
                    validation_data = validation_set
                    )

Epoch 1/10
38/38 [==============================] - 33s 660ms/step - loss: 0.3882 - acc: 0.8274 - val_loss: 9.0517 - val_acc: 0.6880
Epoch 2/10
38/38 [==============================] - 24s 609ms/step - loss: 0.2815 - acc: 0.8783 - val_loss: 3.3731 - val_acc: 0.7032
Epoch 3/10
38/38 [==============================] - 24s 612ms/step - loss: 0.2115 - acc: 0.9103 - val_loss: 3.3779 - val_acc: 0.7285
Epoch 4/10
38/38 [==============================] - 24s 610ms/step - loss: 0.1868 - acc: 0.9187 - val_loss: 2.9385 - val_acc: 0.6408
Epoch 5/10
38/38 [==============================] - 24s 608ms/step - loss: 0.1508 - acc: 0.9444 - val_loss: 1.9677 - val_acc: 0.6594
Epoch 6/10
38/38 [==============================] - 24s 611ms/step - loss: 0.1697 - acc: 0.9360 - val_loss: 1.4872 - val_acc: 0.7167
Epoch 7/10
38/38 [==============================] - 24s 609ms/step - loss: 0.1623 - acc: 0.9436 - val_loss: 0.6498 - val_acc: 0.8583
Epoch 8/10
38/38 [==============================] - 24s 614ms/step - 

In [ ]:
model.save("/content/drive/Shareddrives/SEAI Project/Models/InceptionV3.h5")

In [ ]:
model = keras.models.load_model('/content/drive/Shareddrives/SEAI Project/Models/InceptionV3.h5')

In [ ]:
model.evaluate(test_set)

135/135 [==============================] - 39s 200ms/step - loss: 0.6279 - acc: 0.6324


[0.6279073357582092, 0.6323546767234802]